In [2]:
!nvidia-smi

Tue Jun 15 18:51:13 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.51.06    Driver Version: 450.51.06    CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:3B:00.0 Off |                    0 |
| N/A   33C    P0    39W / 300W |      9MiB / 32510MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  Off  | 00000000:89:00.0 Off |                    0 |
| N/A   

In [3]:
import sys
sys.path.append("/pfs/data5/home/st/st_us-051200/st_st169719/third_party")
print(sys.path)

['', '/home/st/st_us-051200/st_st169719/.local/lib/python3.6/site-packages', '/opt/bwhpc/common/jupyter/base/lib/python3.6/site-packages', '/usr/lib64/python36.zip', '/usr/lib64/python3.6', '/usr/lib64/python3.6/lib-dynload', '/pfs/data5/software_uc2/bwhpc/common/jupyter/base/lib64/python3.6/site-packages', '/pfs/data5/software_uc2/bwhpc/common/jupyter/base/lib/python3.6/site-packages', '/usr/lib64/python3.6/site-packages', '/usr/lib/python3.6/site-packages', '/opt/bwhpc/common/jupyter/base/lib/python3.6/site-packages/IPython/extensions', '/pfs/data5/home/st/st_us-051200/st_st169719/.ipython', '/pfs/data5/home/st/st_us-051200/st_st169719/third_party']


In [4]:
from typing import Dict
import csv
import lineflow as lf
import json_lines
from functools import partial
import torch
from torch.utils.data import DataLoader, SequentialSampler, RandomSampler
from transformers import AlbertForMultipleChoice, AlbertTokenizer, AdamW

In [4]:
!pwd

/pfs/data5/home/st/st_us-051200/st_st169719/Cosmos/script


In [5]:
!cd dataset && wget https://raw.githubusercontent.com/wilburOne/cosmosqa/master/data/valid.csv
!cd dataset && wget https://raw.githubusercontent.com/wilburOne/cosmosqa/master/data/train.csv

--2021-06-12 20:43:03--  https://raw.githubusercontent.com/wilburOne/cosmosqa/master/data/valid.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2128345 (2.0M) [text/plain]
Saving to: ‘valid.csv’

valid.csv           100%[===================>]   2.03M  --.-KB/s    in 0.07s   

2021-06-12 20:43:03 (29.6 MB/s) - ‘valid.csv’ saved [2128345/2128345]

--2021-06-12 20:43:04--  https://raw.githubusercontent.com/wilburOne/cosmosqa/master/data/train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16660449 (16M) [text/plain]
Saving t

In [5]:
import torch
torch.manual_seed(0)
import random
random.seed(0)
import numpy as np
np.random.seed(0)

In [6]:
def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

In [7]:
val_datas = []
with open('../dataset/valid.csv', 'r') as file:
    reader = csv.reader(file)
    next(reader)
    for sample in reader:
      _id = sample[0]
      _article = sample[1]
      _question = sample[2]
      _options = []
      _answer = sample[-1]
      _options.append(sample[3])
      _options.append(sample[4])
      _options.append(sample[5])
      _options.append(sample[6])

      data = {
              "id": _id,
              "article": _article,
              "options": _options,
              "question": _question,
              "answer": _answer
              }
      val_datas.append(data)
val_dataset = lf.Dataset(val_datas)

In [8]:
train_datas = []
with open('../dataset/train.csv', 'r') as file:
    reader = csv.reader(file)
    next(reader)
    for sample in reader:
      _id = sample[0]
      _article = sample[1]
      _question = sample[2]
      _options = []
      _answer = sample[-1]
      _options.append(sample[3])
      _options.append(sample[4])
      _options.append(sample[5])
      _options.append(sample[6])

      data = {
              "id": _id,
              "article": _article,
              "options": _options,
              "question": _question,
              "answer": _answer
              }
      train_datas.append(data)
train_dataset = lf.Dataset(train_datas)

In [9]:
MAX_LEN = 1024
NUM_LABELS = 4
label_map = {"A": 0, "B": 1, "C": 2, "D": 3}
BATCH_SIZE = 32

In [10]:
def preprocess(tokenizer: AlbertTokenizer, x: Dict) -> Dict:

    choices_features = []

    option: str
    for option in x["options"]:
        text_a = x["article"]
        text_b = x["question"] + " " + option

        inputs = tokenizer.encode_plus(
                text_a,
                text_b,
                add_special_tokens=True,
                max_length=MAX_LEN
                )
        input_ids, token_type_ids = inputs["input_ids"], inputs["token_type_ids"]
        attention_mask = [1] * len(input_ids)

        pad_token_id = tokenizer.pad_token_id
        padding_length = MAX_LEN - len(input_ids)
        input_ids = input_ids + ([pad_token_id] * padding_length)
        attention_mask = attention_mask + ([0] * padding_length)
        token_type_ids = token_type_ids + ([pad_token_id] * padding_length)

        assert len(input_ids) == MAX_LEN, "Error with input length {} vs {}".format(len(input_ids), MAX_LEN)
        assert len(attention_mask) == MAX_LEN, "Error with input length {} vs {}".format(len(attention_mask), MAX_LEN)
        assert len(token_type_ids) == MAX_LEN, "Error with input length {} vs {}".format(len(token_type_ids), MAX_LEN)

        choices_features.append({
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "token_type_ids": token_type_ids,
            })

    labels = int(x["answer"])
    label = torch.tensor(labels).long()

    return {
            "id": x["id"],
            "label": label,
            "input_ids": torch.tensor([cf["input_ids"] for cf in choices_features]),
            "attention_mask": torch.tensor([cf["attention_mask"] for cf in choices_features]),
            "token_type_ids": torch.tensor([cf["token_type_ids"] for cf in choices_features]),
            }


def get_dataloader(tokenizer):
    preprocessor = partial(preprocess, tokenizer)

    train_dataloader = DataLoader(
            train_dataset.map(preprocessor).save("/pfs/data5/home/st/st_us-051200/st_st169719/Cosmos/dataset/CacheFiles/MAX_LEN1024/BatchSize32/train_cosmos.cache"),
            sampler=RandomSampler(train_dataset),
            batch_size=BATCH_SIZE,
            worker_init_fn=seed_worker,
            num_workers=40
            )

    val_dataloader = DataLoader(
            val_dataset.map(preprocessor).save("/pfs/data5/home/st/st_us-051200/st_st169719/Cosmos/dataset/CacheFiles/MAX_LEN1024/BatchSize32/val_cosmos.cache"),
            sampler=SequentialSampler(val_dataset),
            batch_size=BATCH_SIZE,
            worker_init_fn=seed_worker,
            num_workers=40
            )

    return train_dataloader, val_dataloader

In [11]:
tokenizer = AlbertTokenizer.from_pretrained("albert-base-v2", do_lower_case=True)

In [12]:
train_dataloader, val_dataloader = get_dataloader(tokenizer)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Saving data to /pfs/data5/home/st/st_us-051200/st_st169719/Cosmos/dataset/CacheFiles/BatchSize32/train_cosmos.cache...
Saving data to /pfs/data5/home/st/st_us-051200/st_st169719/Cosmos/dataset/CacheFiles/BatchSize32/val_cosmos.cache...
